## Encoder and Decoder networks

## 4. Build the encoder and decoder networks
* You should now create the encoder and decoder for the variational autoencoder algorithm.
* You should design these networks yourself, subject to the following constraints:
   * The encoder and decoder networks should be built using the `Sequential` class.
   * The encoder and decoder networks should use probabilistic layers where necessary to represent distributions.
   * The prior distribution should be a zero-mean, isotropic Gaussian (identity covariance matrix).
   * The encoder network should add the KL divergence loss to the model.
* Print the model summary for the encoder and decoder networks.

In [2]:
import math
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_probability as tfp

from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Flatten, Conv2D, UpSampling2D, BatchNormalization, Reshape

tfb = tfp.bijectors
tfd = tfp.distributions
tfpl = tfp.layers

%matplotlib inline

In [3]:
latent_dim = 50

prior = tfd.MultivariateNormalDiag(loc=tf.zeros(latent_dim))

In [4]:
def get_kl_regularizer(prior_distribution):
    """
    This function should create an instance of the KLDivergenceRegularizer 
    according to the above specification. 
    The function takes the prior_distribution, which should be used to define 
    the distribution.
    Your function should then return the KLDivergenceRegularizer instance.
    """
    return tfpl.KLDivergenceRegularizer(prior_distribution,
                                        weight=1.0,
                                        use_exact_kl=False,
                                        test_points_fn=lambda q: q.sample(3),
                                        test_points_reduce_axis=None) 

In [5]:
kl_regularizer = get_kl_regularizer(prior)

In [6]:
event_shape = (36, 36)

encoder = Sequential([
    Conv2D(filters=32, padding='SAME', kernel_size=(4,4), strides=(2,2), activation='relu', input_shape=(36,36,3)),
    BatchNormalization(),
    Conv2D(filters=64, padding='SAME', kernel_size=(4,4), strides=(2,2), activation='relu'),
    BatchNormalization(),
    Conv2D(filters=128, padding='SAME', kernel_size=(4,4), strides=(2,2), activation='relu'),
    BatchNormalization(),
    Conv2D(filters=256, padding='SAME', kernel_size=(4,4), strides=(2,2), activation='relu'),
    BatchNormalization(),
    Flatten(),
    Dense(tfpl.MultivariateNormalTriL.params_size(latent_dim)),
    tfpl.MultivariateNormalTriL(event_size=latent_dim, activity_regularizer=kl_regularizer)
])
encoder.summary()

Instructions for updating:
Do not pass `graph_parents`.  They will  no longer be used.
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 18, 18, 32)        1568      
_________________________________________________________________
batch_normalization (BatchNo (None, 18, 18, 32)        128       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 9, 9, 64)          32832     
_________________________________________________________________
batch_normalization_1 (Batch (None, 9, 9, 64)          256       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 5, 5, 128)         131200    
_________________________________________________________________
batch_normalization_2 (Batch (None, 5, 5, 128)         512       
___________________________________

In [7]:
decoder = Sequential([
    Dense(36*36, activation='relu', input_shape=(latent_dim,)),
    Reshape((9,9,16)),
    UpSampling2D(size=(2,2)),
    Conv2D(filters=128, padding='SAME', kernel_size=(3,3), activation='relu'),
    UpSampling2D(size=(2,2)),
    Conv2D(filters=128, padding='SAME', kernel_size=(3,3), activation='relu'),
    Conv2D(filters=3, padding='SAME', kernel_size=(3,3)),
    Flatten(),
    tfpl.IndependentBernoulli(event_shape=(36,36,3))
])
decoder.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 1296)              66096     
_________________________________________________________________
reshape (Reshape)            (None, 9, 9, 16)          0         
_________________________________________________________________
up_sampling2d (UpSampling2D) (None, 18, 18, 16)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 18, 18, 128)       18560     
_________________________________________________________________
up_sampling2d_1 (UpSampling2 (None, 36, 36, 128)       0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 36, 36, 128)       147584    
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 36, 36, 3)        

In [8]:
vae = Model(inputs=encoder.inputs, outputs=decoder(encoder.outputs))